# Data Science Dashboard

In [1]:
# !pip install plotly
# !pip install plotly==5.11.0
# !pip install dash
# !pip install dash-bootstrap-components

import plotly
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
import numpy as np

import dash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc

from dash.dependencies import Input,Output

In [2]:
df=pd.read_csv(r"ds_salaries.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          607 non-null    int64  
 1   work_year           607 non-null    int64  
 2   experience_level    607 non-null    object 
 3   employment_type     607 non-null    object 
 4   job_title           607 non-null    object 
 5   salary              607 non-null    int64  
 6   salary_currency     607 non-null    object 
 7   salary_in_usd       607 non-null    int64  
 8   employee_residence  607 non-null    object 
 9   remote_ratio        607 non-null    int64  
 10  company_location    607 non-null    object 
 11  company_size        607 non-null    object 
 12  Continent           607 non-null    object 
 13  Country             607 non-null    object 
 14  Alpha-3 code        607 non-null    object 
 15  Lat                 607 non-null    float64
 16  Long    

# Data Cleaning

In [3]:
# Drop irrelevant columns
df.drop(columns=['salary','salary_currency','Unnamed: 0'],inplace=True)

#Rename Column headers
df.rename(columns={'salary_in_usd':'Salary',
                   'work_year':'Year',
                   'experience_level':'Experience',
                   'job_title':'Job_Title',
                   'company_size':'Company_Size','employment_type':'Employment'},inplace=True)

#Replace Experience Level Legend with values
df['Experience'].replace({'MI':'Middle Level','SE':'Senior Level','EX':'Executive Level','EN':'Entry Level'},inplace=True)
df['Employment'].replace({'FT':'Full Time','PT':'Part Time','CT':'Contractual','FL':'Freelance'},inplace=True)
df['Company_Size'].replace({'L':'Large (10,000+)','S':'Small (0-1,000)','M':'Medium (1,000-10,000)'},inplace=True)


df.head()

,Year,Experience,Employment,Job_Title,Salary,employee_residence,remote_ratio,company_location,Company_Size,Continent,Country,Alpha-3 code,Lat,Long
0,2020,Middle Level,Full Time,Data Scientist,79833,DE,0,DE,"Large (10,000+)",Europe,Germany,DEU,51.0,9.0
1,2020,Senior Level,Full Time,Machine Learning Scientist,260000,JP,0,JP,"Small (0-1,000)",Asia,Japan,JPN,36.0,138.0
2,2020,Senior Level,Full Time,Big Data Engineer,109024,GB,50,GB,"Medium (1,000-10,000)",Europe,United Kingdom,GBR,54.0,-2.0
3,2020,Middle Level,Full Time,Product Data Analyst,20000,HN,0,HN,"Small (0-1,000)",Americas,Honduras,HND,15.0,-86.5
4,2020,Senior Level,Full Time,Machine Learning Engineer,150000,US,50,US,"Large (10,000+)",Americas,United States,USA,38.0,-97.0


In [ ]:
# Framework with 4 filers : Country, Company Size, Experience Level, Work Mode

app = dash.Dash(__name__,external_stylesheets=[dbc.themes.CERULEAN])

app.layout = dbc.Container([
    
    ##################### -- TITLE -- #####################
    dbc.Row(dbc.Col(html.H1("Data Science Dashboard with Dash",
                             className="text-center font-weight-bold mb-4"),
                    width=12
                    )
           ),
    
    ##################### -- FILTERS -- #####################
    # Row for Filters
    dbc.Row([
            dbc.Col(dcc.Dropdown(id='FL_Country',
                                  options=[{'label':'All Countries','value':'All'},
                                           {'label':'United States','value':'United States'},
                                           {'label':'India','value':'India'}],
                                  value='All',
                                  placeholder='Country'),width=2
                     ),
             dbc.Col(dcc.Dropdown(id='FL_Company_Size',
                                  options=[{'label':'All Companies','value':'All'},
                                           {'label':'Small','value':'Small (0-1,000)'},
                                           {'label':'Medium','value':'Medium (1,000-10,000)'},
                                           {'label':'Large','value':'Large (10,000+)'}],
                                  value='All',
                                  placeholder='Company Size'),width=2
                    ),
             dbc.Col(dcc.Dropdown(id='FL_Experience',
                                  options=[{'label':'All Experience Level','value':'All'},
                                           {'label':'Entry Level','value':'Entry Level'},
                                           {'label':'Middle Level','value':'Middle Level'},
                                           {'label':'Senior Level','value':'Senior Level'},
                                           {'label':'Executive Level','value':'Executive Level'}],
                                  value='All',
                                  placeholder='Select Experience Level...'),width=2
                    ),
         dbc.Col(dcc.Checklist(id='FL_Year',
                                  options=[{'label':'2020','value':2020},
                                           {'label':'2021','value':2021},
                                           {'label':'2021','value':2022}],
                                  value=[2020,2021,2022]),width=2
                    )
            ],justify='around'
           ),
    
    ##################### -- CARDS -- #####################
#     Row for Cards
    dbc.Row([dbc.Col(dbc.Card([#dbc.CardImg(src="/assets/cityicon.jpg",top=True,) ,
                dbc.CardHeader('Average Salary'), 
                dbc.CardBody(html.H4(id='card1',children=[]))
               ],style={"width": "1"}),width=2),
             
             dbc.Col(dbc.Card([#dbc.CardImg(src="/assets/cityicon.jpg",top=True,) ,
                dbc.CardHeader('Top Paying Location'), 
                dbc.CardBody(html.H4(id='card2',children=[]))
               ],style={"width": "1"}),width=2),
             
             dbc.Col(dbc.Card([#dbc.CardImg(src="/assets/cityicon.jpg",top=True,) ,
                dbc.CardHeader('Most Common Title'), 
                dbc.CardBody(html.H4(id='card3',children=[]))
               ],style={"width": "1"}),width=2),
             
             dbc.Col(dbc.Card([#dbc.CardImg(src="/assets/cityicon.jpg",top=True,) ,
                dbc.CardHeader('Percentage of Jobs Remote'), 
                dbc.CardBody(html.H4(id='card4',children=[]))
               ],style={"width": "1"}),width=2)
             
            ],justify='around',
           ),
    
    ##################### -- GRAPHS -- #####################
    # Row1 for Graphs
    dbc.Row([dbc.Col(dcc.Graph(id='graph1',figure={}),width=4),
             dbc.Col(dcc.Graph(id='graph2',figure={}),width=4),
             dbc.Col(dcc.Graph(id='graph3',figure={}),width=4)
            ],justify='start'
           ),
    
    # Row2 for Graphs
    dbc.Row([dbc.Col(dcc.Graph(id='graph4',figure={}),width=5),
             dbc.Col(dcc.Graph(id='graph5',figure={}),width=5)
            ],justify='around'
           )
    
],fluid=True
)
@app.callback(
    [Output(component_id='graph1',component_property='figure'),
    Output(component_id='graph2',component_property='figure'),
    Output(component_id='graph3',component_property='figure'),
    Output(component_id='graph4',component_property='figure'),
    Output(component_id='graph5',component_property='figure'),
    Output(component_id='card1',component_property='children'),
    Output(component_id='card2',component_property='children'),
    Output(component_id='card3',component_property='children'),
    Output(component_id='card4',component_property='children')],
    [Input(component_id='FL_Country',component_property='value'),
    Input(component_id='FL_Company_Size',component_property='value'),
    Input(component_id='FL_Experience',component_property='value'),
    Input(component_id='FL_Year',component_property='value')]
)
def update_graph(filter_country,filter_company,filter_experience,filter_year):
    print('Filter for: ',filter_country,'-',filter_company,'-',filter_experience,'-',filter_year)
    
    df_copy=df.copy()
    if len(filter_year)!=3:
        df_copy=df_copy[df_copy['Year'].isin(filter_year)]
    if filter_experience!='All':
        df_copy=df_copy[df_copy['Experience']==filter_experience]
    if filter_company!='All':
        df_copy=df_copy[df_copy['Company_Size']==filter_company]
    if filter_country!='All':
        df_copy=df_copy[df_copy['Country']==filter_country]
    
    ##########################################
    fig_1 = px.histogram(df_copy, x="Salary",
                   color='Company_Size',
                   title='Salary Distribution Across Company Size',
                   labels={'Salary':'Salary(in US$)','count':'Count of People'},
                   opacity=0.75,
                   color_discrete_sequence=['#1da1f2','#2774AE','#120A8F'],marginal='violin')
    fig_1.update_layout(showlegend=True)
    fig_1.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})
    
    ##########################################
    df2=df_copy.groupby(by=['Alpha-3 code','Country','Continent','Lat','Long'],as_index=False)['Salary'].mean()
    df2['Salary']=df2['Salary'].round()
    
    fig_2=px.choropleth(df2,locations='Alpha-3 code',
                   color='Salary',
                    hover_data=['Country','Salary'],
                   hover_name='Country',
                    title='Average Salary Across Countries',
                  color_continuous_scale=px.colors.sequential.GnBu_r,basemap_visible=True,projection='robinson')
    fig_2.update_layout(margin=dict(l=10, r=10, t=30, b=50))
    
    ##########################################
    df3=df_copy.groupby(by=['Continent'],as_index=False)['Salary'].mean()
    df3['Salary']=round(df3['Salary'])
    df3.sort_values('Salary',inplace=True)
    
    fig_3 = px.bar(df3, x="Salary", y="Continent", orientation='h',color='Salary',
              title="Salary Distribution by Region",
              labels={'Salary':'Salary in US$'},
              opacity=0.75,
              color_continuous_scale=px.colors.sequential.GnBu
             )
    fig_3.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})
    
    ##########################################
    df4=df_copy.groupby(by=['Experience'],as_index=False)['Salary'].mean()
    df4['Salary']=round(df4['Salary'])
    df4.sort_values('Salary',inplace=True,ascending=False)

    fig_4=px.bar(df4,x='Experience',y='Salary',
            title='Salary Distribution by Experience Level',
#             pattern_shape='Experience',
            opacity=0.75,color_continuous_scale=px.colors.sequential.GnBu,
            color='Salary',
            text='Salary',
            labels={'Salary':'Salary in USD','Experience':'Experience Level'})

    fig_4.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})
    
    ##########################################
    df5=df_copy.groupby(by=['Job_Title'],as_index=False)['Salary'].mean()
    df5['Salary']=round(df5['Salary'])
    df5.sort_values('Salary',inplace=True,ascending=False)
    
    fig_5 = px.bar(df5[0:10], x="Salary", y="Job_Title", orientation='h',color='Salary',
              color_continuous_scale=px.colors.sequential.GnBu,
              title="Top 10 Paying Job Titles"
             )
    fig_5.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})
    
    # Average Salary
    card1='$'+str(round(df_copy['Salary'].mean()))
    # Top Paying Country
    card2= df_copy.groupby(by=['Country'],as_index=False)['Salary'].max().sort_values('Salary',ascending=False).reset_index()['Country'][0]
    # Most Common Job Title
    card3= df_copy.groupby(by=['Job_Title'],as_index=False)['Salary'].count().sort_values('Salary',ascending=False).reset_index()['Job_Title'][0]
    # Percentage of Jobs Remote
    card4= str(round((df_copy[df_copy['remote_ratio']>0]['remote_ratio'].count()/df_copy['remote_ratio'].count())*100,2))+' %'
    
    return [fig_1,fig_2,fig_3,fig_4,fig_5,card1,card2,card3,card4]

if __name__=='__main__':
    app.run_server(debug=False)
    

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jul/2023 00:01:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2023 00:01:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2023 00:01:04] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2023 00:01:04] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2023 00:01:04] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2023 00:01:04] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


Filter for:  All - All - All - [2020, 2021, 2022]


127.0.0.1 - - [03/Jul/2023 00:01:07] "POST /_dash-update-component HTTP/1.1" 200 -


Filter for:  All - Small (0-1,000) - All - [2020, 2021, 2022]


...

127.0.0.1 - - [03/Jul/2023 00:01:15] "POST /_dash-update-component HTTP/1.1" 200 -


Filter for:  All - Large (10,000+) - All - [2020, 2021, 2022]


127.0.0.1 - - [03/Jul/2023 00:01:19] "POST /_dash-update-component HTTP/1.1" 200 -


Filter for:  All - All - All - [2020, 2021, 2022]


127.0.0.1 - - [03/Jul/2023 00:01:22] "POST /_dash-update-component HTTP/1.1" 200 -


# Import links
https://bootswatch.com/default/ <br>
https://hackerthemes.com/bootstrap-cheatsheet/#mb-1 <br>
https://dash-bootstrap-components.opensource.faculty.ai/docs/components/card/ <br>
https://www.youtube.com/watch?v=0mfIK8zxUds

# GraphPlots

### 1. Salary Historgram

In [ ]:
# fig = px.histogram(df, x="salary_in_usd",title="Salary Distribution",color=None)
# To make color WHITE and remove background
# fig.update_layout({'paper_bgcolor': 'rgba(0,0,0,0)','font_color':'#FFFFFF'})
# fig.update_layout(title='Salary Distribution Across Company Size')


fig_1 = px.histogram(df, x="Salary",
                   color='Company_Size',
                   title='Salary Distribution Across Company Size',
                   labels={'Salary':'Salary(in US$)','count':'Count of People'},
                   opacity=0.75,
                   color_discrete_sequence=['#1da1f2','#2774AE','#120A8F'],marginal='violin')
fig_1.update_layout(showlegend=False)
fig_1.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})

fig_1.show()

### 2. Bubble Chart

In [ ]:
df1=df.groupby(by=['Alpha-3 code','Country','Continent','Lat','Long'],as_index=False)['Salary'].mean()
df1['Salary']=df1['Salary'].round()
df1.head()

In [ ]:
# fig_2=px.scatter_mapbox(df1,lat='Lat',lon='Long',color='Continent',
#                         size='Salary',mapbox_style='open-street-map',
#                         zoom=1,
#                         hover_data=['Country','Salary'],hover_name='Country',
#                         size_max=30,opacity=0.7,
#                         title='Avg Salary Distribution across Countries',
#                         text=[item for item in df1['Continent']],
#                         color_discrete_sequence=['#5F9EA0','#7CB9E8','#007FFF','#005A9C','#13274F',])

# fig_2.update_layout(margin=dict(l=60, r=60, t=50, b=80))
# fig_2.update_layout(
#         autosize=False,
#         margin = dict(
#                 l=0,
#                 r=0,
#                 b=0,
#                 t=0,
# #                 pad=4,
# #                 autoexpand=True
#             ),
#             width=1000, height=400,
#     )

# fig_2.show()

In [ ]:
fig_2=px.choropleth(df1,locations='Alpha-3 code',
                   color='Salary',
                    hover_data=['Country','Salary'],
                   hover_name='Country',
                    title='Average Salary Across Countries',
                  color_continuous_scale=px.colors.sequential.GnBu_r,basemap_visible=True,projection='robinson')
fig_2.update_layout(margin=dict(l=10, r=10, t=30, b=50))
fig_2.show()

### By Region

In [ ]:
df2=df.groupby(by=['Continent'],as_index=False)['Salary'].mean()
df2['Salary']=round(df2['Salary'])
df2.sort_values('Salary',inplace=True)
df2

In [ ]:
fig_3 = px.bar(df2, x="Salary", y="Continent", orientation='h',color='Salary',
              title="Salary Distribution by Region",
              labels={'Salary':'Salary in US$'},
              opacity=0.75,
              color_continuous_scale=px.colors.sequential.GnBu
             )
fig_3.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})
fig_3.show()

### 5. By Experience

In [ ]:
df3=df.groupby(by=['Experience'],as_index=False)['Salary'].mean()
df3['Salary']=round(df3['Salary'])
df3.sort_values('Salary',inplace=True,ascending=False)
df3

In [ ]:
fig_4=px.bar(df3,x='Experience',y='Salary',
            title='Salary Distribution by Experience Level',
#             pattern_shape='Experience',
            opacity=0.75,color_continuous_scale=px.colors.sequential.GnBu,
            color='Salary',
            text='Salary',
            labels={'Salary':'Salary in USD','Experience':'Experience Level'})

fig_4.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})
fig_4.show()

### 5. Top N job Title 

In [ ]:
df4=df.groupby(by=['Job_Title'],as_index=False)['Salary'].mean()
df4['Salary']=round(df4['Salary'])
df4.sort_values('Salary',inplace=True,ascending=False)

In [ ]:
fig_5 = px.bar(df4[0:10], x="Salary", y="Job_Title", orientation='h',color='Salary',
              color_continuous_scale=px.colors.sequential.GnBu
#               title="Top 10 Paying Job Titles"
             )
fig_5.update_layout()
fig_5.show()

In [ ]:
# Card 1 : Mean Salary
card1=dbc.Card([#dbc.CardImg(src="/assets/cityicon.jpg",top=True,) ,
                dbc.CardHeader('Average Salary'), 
                dbc.CardBody(html.H4('$20,000'))
               ],style={"width": "1"})

card2=dbc.Card([#dbc.CardImg(src="/assets/cityicon.jpg",top=True,) ,
                dbc.CardHeader('Top Paying Location'), 
                dbc.CardBody(html.H4('United States'))
               ],style={"width": "1"})

card3=dbc.Card([#dbc.CardImg(src="/assets/cityicon.jpg",top=True,) ,
                dbc.CardHeader('Most Common Title'), 
                dbc.CardBody(html.H4('Data Scientist'))
               ],style={"width": "1"})

card4=dbc.Card([#dbc.CardImg(src="/assets/cityicon.jpg",top=True,) ,
                dbc.CardHeader('Remote Job'), 
                dbc.CardBody(html.H4('50%'))
               ],style={"width": "1"})